In [1]:
import data_from_database_mk2 as dfd
import nltk
import numpy
import pandas
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Mengambil data dari database
dataset_document = dfd.get_data()

def wordProcessing(content_article):
    """
    Fungsi untuk menghitung 'word' dalam suatu artikel
    """
    tokens = word_tokenize(re.sub('[^ 0-9a-z]+', ' ', content_article.lower())) # Menghilangkan tanda baca   
    english_stopwords = stopwords.words('english') # Menghilangkan stop words
    english_stopwords.extend(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'])
    tokens_wo_stopwords = [t for t in tokens if t not in english_stopwords]

    # Menghitung banyaknya tokens
    freq = nltk.FreqDist(tokens_wo_stopwords)
    word_document_dictionary = {}

    # Menampung token tersebut dalam format dictionary
    for word, count in freq.most_common(999999):
        word_document_dictionary.update({word: count})
    
    return word_document_dictionary

def documentWordProcessing(content_article, title_article):
    """
    Fungsi untuk membuat dataframe antara 'document' dengan 'word'
    """
    word_document_dictionary = wordProcessing(content_article)
    datafr = pandas.DataFrame(word_document_dictionary,
        index=[title_article]
    )
    return datafr


In [2]:
dataset_document

(('alvin bragg',
  'While pinning Trump case to campaign laws, Bragg eyes a backup plan | The Hill',
  '\n\nThe much-awaited charges against Donald Trump show Manhattan District Attorney Alvin Bragg (D) plans to largely rest on campaign laws to prosecute the former president for obscuring his reimbursement of hush money payments.\nBut sprinkled into charging documents and public statements from Bragg are references to tax law violations — a sign New York prosecutors may be hedging their bets by bringing a broader\xa0case against the former president.\n\n\n  \n\nTrump was charged on 34 felony counts during his arraignment Tuesday, each stemming from an arrangement Trump made with fixer Michael Cohen after he\xa0made a $130,000 payment\xa0to adult film star Stormy Daniels.\nThe case is built on falsification of business records charges, with prosecutors arguing the “unlawful scheme” violated election laws.\n“The defendant orchestrated a scheme with others to influence the 2016 presidenti

In [ ]:
def document_processing(dataset_document):  
  title_before = dataset_document[0][1]
  dataframe_document_word = []
  dataframe_document_word.append(documentWordProcessing(dataset_document[0][2], dataset_document[0][1]))
  
  tag_dictionary = {}
  # Tempat untuk menghitung banyaknya tag dalam suatu dokumen
  dataframe_document_tag = []
  
  for data in dataset_document:
    # Jika judul dokumen berbeda dengan row sebelumnya
    if title_before != data[1]:
      datafr = pandas.DataFrame(tag_dictionary,
          index=[title_before]
      )
      dataframe_document_tag.append(datafr)
      dataframe_document_word.append(documentWordProcessing(data[2], data[1]))
      title_before = data[1]
      tag_dictionary.clear()
    
    tag_dictionary.update({data[0]: 1})

  datafr = pandas.DataFrame(tag_dictionary,
      index=[title_before]
  )
  dataframe_document_tag.append(datafr)
  tag_dictionary.clear()
  
  return dataframe_document_tag, dataframe_document_word

In [ ]:
dataframe_document_tag, dataframe_document_word = document_processing(dataset_document)

In [ ]:
dataframe_document_tag_join = pandas.concat(dataframe_document_tag)
dataframe_document_tag_join = dataframe_document_tag_join.fillna(0)
dataframe_document_tag_join

In [ ]:
dataframe_document_word_join = pandas.concat(dataframe_document_word)
dataframe_document_word_join = dataframe_document_word_join.fillna(0)
dataframe_document_word_join

In [ ]:

def matrixABtoW(A, B):
    """
    Fungsi untuk memasukkan matriks A, A transpose, B, dan B transpose ke dalam matriks W
    """
    AT = A.transpose()
    BT = B.transpose()

    tag_count, document_count = A.shape
    document_count, word_count = B.shape

    all_count = tag_count + document_count + word_count
    W = numpy.zeros((all_count, all_count))

    # Menempelkan matriks A ke W
    for i in range(tag_count):
        W[i][tag_count:-word_count] = A[i]

    # Menempelkan matriks B Transpose ke W
    for i in range(1, word_count+1):
        W[-i][tag_count:-word_count]= BT[-i]

    # Menempelkan matriks A Transpose ke W
    for i in range(document_count):
        W[tag_count+i][0:tag_count] = AT[i]
        
    # Menempelkan matriks B ke W
    for i in range(document_count):
        W[tag_count+i][-word_count:] = B[i]
    
    return W

# print(dataframe_document_word_join.to_numpy())
# print(dataframe_document_tag_join.to_numpy())

In [ ]:
matrix_w = matrixABtoW(dataframe_document_tag_join.to_numpy().transpose(), dataframe_document_word_join.to_numpy())
matrix_w

In [ ]:
matrix_w.max()